<a href="https://colab.research.google.com/github/GuptaTanay/Battery-Level-Prediction/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import files
# files.upload()

In [2]:
import pandas as pd
from tqdm import tqdm

from scipy.spatial.distance import cdist
from scipy.spatial import distance
from nltk.cluster import KMeansClusterer, euclidean_distance
from scipy import spatial
# from sql import *

In [3]:
fin_data = pd.read_excel('vw_FinancialData_Net_Income.xlsx')
demo_data = pd.read_csv('vw_population_with_Median_Age.csv')

In [4]:
fin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221791 entries, 0 to 221790
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   IDRSSD                221791 non-null  int64         
 1   Cycle_Date            221791 non-null  datetime64[ns]
 2   Cycle Year            221791 non-null  int64         
 3   CU_Bank_Name          221791 non-null  object        
 4   Bank Type             221791 non-null  object        
 5   City_Name             221791 non-null  object        
 6   State_Code            221791 non-null  object        
 7   County_fips           221791 non-null  int64         
 8   County_Name           221686 non-null  object        
 9   TOTAL_ASSETS          221791 non-null  int64         
 10  TOTAL_DEPOSITS        221791 non-null  int64         
 11  TOTAL_LIABILITIES     221791 non-null  int64         
 12  TOTAL_EQUITY_CAPITAL  221791 non-null  int64         
 13 

In [5]:
demo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606676 entries, 0 to 606675
Data columns (total 20 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   CU_BANK_NAME                          606676 non-null  object 
 1   SiteId                                606676 non-null  int64  
 2   Site_Name                             606676 non-null  object 
 3   County_fips                           606676 non-null  int64  
 4   County_Name                           606672 non-null  object 
 5   State_Code                            606676 non-null  object 
 6   Branch_County_FIPS                    606676 non-null  int64  
 7   Branch_County_Name                    606676 non-null  object 
 8   Branch_State_Code                     606676 non-null  object 
 9   Branch_State_Name                     606672 non-null  object 
 10  Branch_City_Name                      606676 non-null  object 
 11  

In [6]:
fin_data.columns

Index(['IDRSSD', 'Cycle_Date', 'Cycle Year', 'CU_Bank_Name', 'Bank Type',
       'City_Name', 'State_Code', 'County_fips', 'County_Name', 'TOTAL_ASSETS',
       'TOTAL_DEPOSITS', 'TOTAL_LIABILITIES', 'TOTAL_EQUITY_CAPITAL',
       'Source_Type', 'Net_Income', 'a', 'Quaterly_Net_Income',
       'Annual_Net_Income', 'ROE', 'ROA'],
      dtype='object')

In [7]:
demo_data.columns

Index(['CU_BANK_NAME', 'SiteId', 'Site_Name', 'County_fips', 'County_Name',
       'State_Code', 'Branch_County_FIPS', 'Branch_County_Name',
       'Branch_State_Code', 'Branch_State_Name', 'Branch_City_Name',
       'Source_Type', 'PhysicalAddressLine1', 'PhysicalAddressPostalCode',
       'Population', 'Year', 'NetFlow', 'Median_age_years',
       'Net_Migration_per_1000_of_Population', 'BranchCount'],
      dtype='object')

In [8]:
latest_fin_data = fin_data.loc[fin_data.Cycle_Date == '3/31/2021']

In [9]:
latest_fin_data.head()

,IDRSSD,Cycle_Date,Cycle Year,CU_Bank_Name,Bank Type,City_Name,State_Code,County_fips,County_Name,TOTAL_ASSETS,TOTAL_DEPOSITS,TOTAL_LIABILITIES,TOTAL_EQUITY_CAPITAL,Source_Type,Net_Income,a,Quaterly_Net_Income,Annual_Net_Income,ROE,ROA
20,1000052,2021-03-31,2021,Washington County Bank,Community Bank,Blair,NE,31177,Washington,522909000,444096000,472564000,50345000,CDR,1405000,0,1405000,6332000,0.125772,0.012109
41,1000089,2021-03-31,2021,HARTFORD MUNICIPAL EMPLOYEES,Credit Union Bank,HARTFORD,CT,9003,Hartford,50413164,47728344,0,47728344,CU,1285257,0,1285257,1213386,0.025422,0.024068
62,1000100,2021-03-31,2021,East Boston Savings Bank,Community Bank,Boston,MA,25025,Suffolk,6501443000,5106059000,5734980000,766463000,CDR,24368000,0,24368000,76681000,0.100045,0.011794
83,1000173,2021-03-31,2021,ALTAONE,Credit Union Bank,Ridgecrest,CA,6029,Kern,719970776,647655714,0,647655714,CU,54116764,0,54116764,53560520,0.082699,0.074392
104,1000276,2021-03-31,2021,The Pittsfield Co-Operative Bank,Community Bank,Pittsfield,MA,25003,Berkshire,379372000,312189000,325823000,53549000,CDR,514000,0,514000,2046000,0.038207,0.005393


In [10]:
# latest_fin_data = latest_fin_data[['CU_Bank_Name', 'Cycle_Date', 'TOTAL_ASSETS']]

In [11]:
# latest_fin_data

In [12]:
latest_fin_data.columns

Index(['IDRSSD', 'Cycle_Date', 'Cycle Year', 'CU_Bank_Name', 'Bank Type',
       'City_Name', 'State_Code', 'County_fips', 'County_Name', 'TOTAL_ASSETS',
       'TOTAL_DEPOSITS', 'TOTAL_LIABILITIES', 'TOTAL_EQUITY_CAPITAL',
       'Source_Type', 'Net_Income', 'a', 'Quaterly_Net_Income',
       'Annual_Net_Income', 'ROE', 'ROA'],
      dtype='object')

In [13]:
merged_data = latest_fin_data.merge(demo_data, left_on=['CU_Bank_Name', 'City_Name', 'State_Code', 'County_fips', 'County_Name'],
                                    right_on=['CU_BANK_NAME', 'Branch_City_Name','Branch_State_Code', 'Branch_County_FIPS', 'Branch_City_Name'])

In [14]:
merged_data

,IDRSSD,Cycle_Date,Cycle Year,CU_Bank_Name,Bank Type,City_Name,State_Code_x,County_fips_x,County_Name_x,TOTAL_ASSETS,TOTAL_DEPOSITS,TOTAL_LIABILITIES,TOTAL_EQUITY_CAPITAL,Source_Type_x,Net_Income,a,Quaterly_Net_Income,Annual_Net_Income,ROE,ROA,CU_BANK_NAME,SiteId,Site_Name,County_fips_y,County_Name_y,State_Code_y,Branch_County_FIPS,Branch_County_Name,Branch_State_Code,Branch_State_Name,Branch_City_Name,Source_Type_y,PhysicalAddressLine1,PhysicalAddressPostalCode,Population,Year,NetFlow,Median_age_years,Net_Migration_per_1000_of_Population,BranchCount
0,1000584,2021-03-31,2021,GREENVILLE,Credit Union Bank,Greenville,SC,45045,Greenville,359320995,321630018,0,321630018,CU,30326961,0,30326961,31901359,0.099186,0.088782,GREENVILLE,19975,Main Corporate Office,45045,Greenville,SC,45045,Greenville,SC,South Carolina,Greenville,CU,1501 Wade Hampton Blvd,29609,482191,2016,2729.0,37.9,5.659,20
1,1000584,2021-03-31,2021,GREENVILLE,Credit Union Bank,Greenville,SC,45045,Greenville,359320995,321630018,0,321630018,CU,30326961,0,30326961,31901359,0.099186,0.088782,GREENVILLE,19975,Main Corporate Office,45045,Greenville,SC,45045,Greenville,SC,South Carolina,Greenville,CU,1501 Wade Hampton Blvd,29609,490332,2017,3971.0,38.0,8.098,20
2,1000584,2021-03-31,2021,GREENVILLE,Credit Union Bank,Greenville,SC,45045,Greenville,359320995,321630018,0,321630018,CU,30326961,0,30326961,31901359,0.099186,0.088782,GREENVILLE,19975,Main Corporate Office,45045,Greenville,SC,45045,Greenville,SC,South Carolina,Greenville,CU,1501 Wade Hampton Blvd,29609,498402,2018,5263.0,38.1,10.559,20
3,1000584,2021-03-31,2021,GREENVILLE,Credit Union Bank,Greenville,SC,45045,Greenville,359320995,321630018,0,321630018,CU,30326961,0,30326961,31901359,0.099186,0.088782,GREENVILLE,19975,Main Corporate Office,45045,Greenville,SC,45045,Greenville,SC,South Carolina,Greenville,CU,1501 Wade Hampton Blvd,29609,507003,2019,0.0,38.2,0.000,20
4,1000584,2021-03-31,2021,GREENVILLE,Credit Union Bank,Greenville,SC,45045,Greenville,359320995,321630018,0,321630018,CU,30326961,0,30326961,31901359,0.099186,0.088782,GREENVILLE,19986,Mauldin Branch,45045,Greenville,SC,45045,Greenville,SC,South Carolina,Greenville,CU,142 Tanner Rd,29607,482191,2016,2729.0,37.9,5.659,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9207,996260,2021-03-31,2021,Ctbc Bank Corp. (Usa),Community Bank,Los Angeles,CA,6037,Los Angeles,3969104000,3334802000,3399079000,570025000,CDR,12903000,0,12903000,40726000,0.071445,0.010260,Ctbc Bank Corp. (Usa),13219,"801 South Figueroa Street, Suite 2300",6037,Los Angeles,CA,6037,Los Angeles,CA,California,Los Angeles,CDR,"801 South Figueroa Street, Suite 2300",90017,10081570,2019,0.0,36.5,0.000,28
9208,999994,2021-03-31,2021,THE FINEST,Credit Union Bank,New York,NY,36061,New York,15488636,14246402,0,14246402,CU,1218590,0,1218590,1577019,0.110695,0.101817,THE FINEST,5,Main Branch - Police Plaza,36061,New York,NY,36061,New York,NY,New York,New York,CU,215 Park Row,10038,1634989,2016,7884.0,36.7,4.822,4
9209,999994,2021-03-31,2021,THE FINEST,Credit Union Bank,New York,NY,36061,New York,15488636,14246402,0,14246402,CU,1218590,0,1218590,1577019,0.110695,0.101817,THE FINEST,5,Main Branch - Police Plaza,36061,New York,NY,36061,New York,NY,New York,New York,CU,215 Park Row,10038,1653877,2017,7947.0,37.1,4.805,4
9210,999994,2021-03-31,2021,THE FINEST,Credit Union Bank,New York,NY,36061,New York,15488636,14246402,0,14246402,CU,1218590,0,1218590,1577019,0.110695,0.101817,THE FINEST,5,Main Branch - Police Plaza,36061,New York,NY,36061,New York,NY,New York,New York,CU,215 Park Row,10038,1632480,2018,1770.0,37.3,1.084,4


In [15]:
def find_groups(df):
  if df.TOTAL_ASSETS<= 2 * (10**9):
    return 'group_0'
  elif df.TOTAL_ASSETS> 2 * (10**9) and df.TOTAL_ASSETS <= 5 * (10**9):
    return 'group_1'
  elif df.TOTAL_ASSETS> 5 * (10**9) and df.TOTAL_ASSETS<= 10 * (10**9):
    return 'group_2'
  else:
    return 'group_3'

In [16]:
latest_fin_data['GroupName'] = latest_fin_data.apply(find_groups, axis =1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
latest_fin_data

,IDRSSD,Cycle_Date,Cycle Year,CU_Bank_Name,Bank Type,City_Name,State_Code,County_fips,County_Name,TOTAL_ASSETS,TOTAL_DEPOSITS,TOTAL_LIABILITIES,TOTAL_EQUITY_CAPITAL,Source_Type,Net_Income,a,Quaterly_Net_Income,Annual_Net_Income,ROE,ROA,GroupName
20,1000052,2021-03-31,2021,Washington County Bank,Community Bank,Blair,NE,31177,Washington,522909000,444096000,472564000,50345000,CDR,1405000,0,1405000,6332000,0.125772,0.012109,group_0
41,1000089,2021-03-31,2021,HARTFORD MUNICIPAL EMPLOYEES,Credit Union Bank,HARTFORD,CT,9003,Hartford,50413164,47728344,0,47728344,CU,1285257,0,1285257,1213386,0.025422,0.024068,group_0
62,1000100,2021-03-31,2021,East Boston Savings Bank,Community Bank,Boston,MA,25025,Suffolk,6501443000,5106059000,5734980000,766463000,CDR,24368000,0,24368000,76681000,0.100045,0.011794,group_2
83,1000173,2021-03-31,2021,ALTAONE,Credit Union Bank,Ridgecrest,CA,6029,Kern,719970776,647655714,0,647655714,CU,54116764,0,54116764,53560520,0.082699,0.074392,group_0
104,1000276,2021-03-31,2021,The Pittsfield Co-Operative Bank,Community Bank,Pittsfield,MA,25003,Berkshire,379372000,312189000,325823000,53549000,CDR,514000,0,514000,2046000,0.038207,0.005393,group_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221705,999935,2021-03-31,2021,First Century Bank,Community Bank,Tazewell,TN,47025,Claiborne,497436000,450795000,454424000,43012000,CDR,1648000,0,1648000,6165000,0.143332,0.012393,group_0
221727,999992,2021-03-31,2021,LAKOTA,Credit Union Bank,Kyle,SD,46113,Shannon,9370660,6763806,0,6763806,CU,976680,0,976680,840253,0.124227,0.089668,group_0
221748,999993,2021-03-31,2021,FIRST UNITY,Credit Union Bank,MCCOMB,MS,28113,Pike,9071889,8279599,0,8279599,CU,466007,0,466007,716204,0.086502,0.078947,group_0
221769,999994,2021-03-31,2021,THE FINEST,Credit Union Bank,New York,NY,36061,New York,15488636,14246402,0,14246402,CU,1218590,0,1218590,1577019,0.110695,0.101817,group_0


In [18]:
latest_fin_data.CU_Bank_Name.value_counts()

First State Bank         43
Farmers State Bank       25
Citizens State Bank      18
Peoples Bank             16
Community State Bank     15
                         ..
POSTAL                    1
NEW ENGLAND TEAMSTERS     1
COLLEGEDALE               1
Alma Bank                 1
SIGNATURE                 1
Name: CU_Bank_Name, Length: 9225, dtype: int64

In [19]:
latest_fin_data.loc[latest_fin_data.CU_Bank_Name == 'First State Bank']

,IDRSSD,Cycle_Date,Cycle Year,CU_Bank_Name,Bank Type,City_Name,State_Code,County_fips,County_Name,TOTAL_ASSETS,TOTAL_DEPOSITS,TOTAL_LIABILITIES,TOTAL_EQUITY_CAPITAL,Source_Type,Net_Income,a,Quaterly_Net_Income,Annual_Net_Income,ROE,ROA,GroupName
6606,10849,2021-03-31,2021,First State Bank,Community Bank,Lonoke,AR,5085,Lonoke,266749000,237202000,237763000,28986000,CDR,696000,0,696000,1956000,0.067480,0.007332,group_0
7919,113740,2021-03-31,2021,First State Bank,Community Bank,Eastpointe,MI,26099,Macomb,1000660000,909211000,910673000,89987000,CDR,1467000,0,1467000,6909000,0.076777,0.006904,group_0
8301,114840,2021-03-31,2021,First State Bank,Community Bank,Russellville,AR,5115,Pope,412808000,364733000,375529000,37279000,CDR,1360000,0,1360000,3699000,0.099224,0.008960,group_0
8811,116554,2021-03-31,2021,First State Bank,Community Bank,Elmore City,OK,40049,Garvin,15469000,9326000,13003000,2466000,CDR,-66000,0,-66000,385000,0.156123,0.024888,group_0
18596,151957,2021-03-31,2021,First State Bank,Community Bank,Gothenburg,NE,31047,Dawson,778109000,557708000,694656000,83453000,CDR,5120000,0,5120000,15923000,0.190802,0.020463,group_0
22660,172859,2021-03-31,2021,First State Bank,Community Bank,Loomis,NE,31137,Phelps,167426000,141150000,145438000,21988000,CDR,492000,0,492000,2670000,0.121429,0.015947,group_0
27087,190947,2021-03-31,2021,First State Bank,Community Bank,Stuart,IA,19077,Guthrie,128386000,116616000,116988000,11398000,CDR,43000,0,43000,291000,0.025530,0.002266,group_0
33447,216164,2021-03-31,2021,First State Bank,Community Bank,Graham,TX,48503,Young,197963000,169797000,179999000,17963000,CDR,1890000,0,1890000,4265000,0.237432,0.021544,group_0
46779,264455,2021-03-31,2021,First State Bank,Community Bank,Athens,TX,48213,Henderson,605844000,541879000,546084000,59760000,CDR,1684000,0,1684000,6530000,0.109270,0.010778,group_0
54283,284958,2021-03-31,2021,First State Bank,Community Bank,Hordville,NE,31081,Hamilton,44313000,37461000,39226000,5088000,CDR,48000,0,48000,201000,0.039504,0.004535,group_0


In [20]:
data_group = latest_fin_data[['CU_Bank_Name', 'GroupName']]

In [21]:
data_group.drop_duplicates(inplace =True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
data_group.CU_Bank_Name.value_counts()

United Bank                    3
Northwest Bank                 3
Peoples Bank                   3
Pinnacle Bank                  3
First Bank                     3
                              ..
GRANITE FURNITURE EMPLOYEES    1
Jackson Savings Bank, SSB      1
N.U.L.                         1
Brazos National Bank           1
SIGNATURE                      1
Name: CU_Bank_Name, Length: 9225, dtype: int64

In [23]:
data_group.loc[data_group.CU_Bank_Name =='Peoples Bank']

,CU_Bank_Name,GroupName
12371,Peoples Bank,group_0
102913,Peoples Bank,group_1
129401,Peoples Bank,group_2


In [24]:
merged_data['GroupName'] = merged_data.apply(find_groups, axis =1)

In [25]:
# merged_data

In [26]:
demo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606676 entries, 0 to 606675
Data columns (total 20 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   CU_BANK_NAME                          606676 non-null  object 
 1   SiteId                                606676 non-null  int64  
 2   Site_Name                             606676 non-null  object 
 3   County_fips                           606676 non-null  int64  
 4   County_Name                           606672 non-null  object 
 5   State_Code                            606676 non-null  object 
 6   Branch_County_FIPS                    606676 non-null  int64  
 7   Branch_County_Name                    606676 non-null  object 
 8   Branch_State_Code                     606676 non-null  object 
 9   Branch_State_Name                     606672 non-null  object 
 10  Branch_City_Name                      606676 non-null  object 
 11  

In [27]:
banks_group0 = merged_data.loc[merged_data.GroupName == 'group_0']
banks_group0

,IDRSSD,Cycle_Date,Cycle Year,CU_Bank_Name,Bank Type,City_Name,State_Code_x,County_fips_x,County_Name_x,TOTAL_ASSETS,TOTAL_DEPOSITS,TOTAL_LIABILITIES,TOTAL_EQUITY_CAPITAL,Source_Type_x,Net_Income,a,Quaterly_Net_Income,Annual_Net_Income,ROE,ROA,CU_BANK_NAME,SiteId,Site_Name,County_fips_y,County_Name_y,State_Code_y,Branch_County_FIPS,Branch_County_Name,Branch_State_Code,Branch_State_Name,Branch_City_Name,Source_Type_y,PhysicalAddressLine1,PhysicalAddressPostalCode,Population,Year,NetFlow,Median_age_years,Net_Migration_per_1000_of_Population,BranchCount,GroupName
0,1000584,2021-03-31,2021,GREENVILLE,Credit Union Bank,Greenville,SC,45045,Greenville,359320995,321630018,0,321630018,CU,30326961,0,30326961,31901359,0.099186,0.088782,GREENVILLE,19975,Main Corporate Office,45045,Greenville,SC,45045,Greenville,SC,South Carolina,Greenville,CU,1501 Wade Hampton Blvd,29609,482191,2016,2729.0,37.9,5.659,20,group_0
1,1000584,2021-03-31,2021,GREENVILLE,Credit Union Bank,Greenville,SC,45045,Greenville,359320995,321630018,0,321630018,CU,30326961,0,30326961,31901359,0.099186,0.088782,GREENVILLE,19975,Main Corporate Office,45045,Greenville,SC,45045,Greenville,SC,South Carolina,Greenville,CU,1501 Wade Hampton Blvd,29609,490332,2017,3971.0,38.0,8.098,20,group_0
2,1000584,2021-03-31,2021,GREENVILLE,Credit Union Bank,Greenville,SC,45045,Greenville,359320995,321630018,0,321630018,CU,30326961,0,30326961,31901359,0.099186,0.088782,GREENVILLE,19975,Main Corporate Office,45045,Greenville,SC,45045,Greenville,SC,South Carolina,Greenville,CU,1501 Wade Hampton Blvd,29609,498402,2018,5263.0,38.1,10.559,20,group_0
3,1000584,2021-03-31,2021,GREENVILLE,Credit Union Bank,Greenville,SC,45045,Greenville,359320995,321630018,0,321630018,CU,30326961,0,30326961,31901359,0.099186,0.088782,GREENVILLE,19975,Main Corporate Office,45045,Greenville,SC,45045,Greenville,SC,South Carolina,Greenville,CU,1501 Wade Hampton Blvd,29609,507003,2019,0.0,38.2,0.000,20,group_0
4,1000584,2021-03-31,2021,GREENVILLE,Credit Union Bank,Greenville,SC,45045,Greenville,359320995,321630018,0,321630018,CU,30326961,0,30326961,31901359,0.099186,0.088782,GREENVILLE,19986,Mauldin Branch,45045,Greenville,SC,45045,Greenville,SC,South Carolina,Greenville,CU,142 Tanner Rd,29607,482191,2016,2729.0,37.9,5.659,20,group_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9203,993371,2021-03-31,2021,PHILADELPHIA LETTER CARRIERS,Credit Union Bank,Philadelphia,PA,42101,Philadelphia,7770080,6821298,0,6821298,CU,691549,0,691549,703468,0.103128,0.090535,PHILADELPHIA LETTER CARRIERS,36084,Philadelphia Letter Carriers FCU,42101,Philadelphia,PA,42101,Philadelphia,PA,Pennsylvania,Philadelphia,CU,2512 S Broad St,19145-4658,1579075,2019,0.0,34.4,0.000,4,group_0
9208,999994,2021-03-31,2021,THE FINEST,Credit Union Bank,New York,NY,36061,New York,15488636,14246402,0,14246402,CU,1218590,0,1218590,1577019,0.110695,0.101817,THE FINEST,5,Main Branch - Police Plaza,36061,New York,NY,36061,New York,NY,New York,New York,CU,215 Park Row,10038,1634989,2016,7884.0,36.7,4.822,4,group_0
9209,999994,2021-03-31,2021,THE FINEST,Credit Union Bank,New York,NY,36061,New York,15488636,14246402,0,14246402,CU,1218590,0,1218590,1577019,0.110695,0.101817,THE FINEST,5,Main Branch - Police Plaza,36061,New York,NY,36061,New York,NY,New York,New York,CU,215 Park Row,10038,1653877,2017,7947.0,37.1,4.805,4,group_0
9210,999994,2021-03-31,2021,THE FINEST,Credit Union Bank,New York,NY,36061,New York,15488636,14246402,0,14246402,CU,1218590,0,1218590,1577019,0.110695,0.101817,THE FINEST,5,Main Branch - Police Plaza,36061,New York,NY,36061,New York,NY,New York,New York,CU,215 Park Row,10038,1632480,2018,1770.0,37.3,1.084,4,group_0


In [28]:
banks_group1 = merged_data.loc[merged_data.GroupName == 'group_1']
banks_group1

,IDRSSD,Cycle_Date,Cycle Year,CU_Bank_Name,Bank Type,City_Name,State_Code_x,County_fips_x,County_Name_x,TOTAL_ASSETS,TOTAL_DEPOSITS,TOTAL_LIABILITIES,TOTAL_EQUITY_CAPITAL,Source_Type_x,Net_Income,a,Quaterly_Net_Income,Annual_Net_Income,ROE,ROA,CU_BANK_NAME,SiteId,Site_Name,County_fips_y,County_Name_y,State_Code_y,Branch_County_FIPS,Branch_County_Name,Branch_State_Code,Branch_State_Name,Branch_City_Name,Source_Type_y,PhysicalAddressLine1,PhysicalAddressPostalCode,Population,Year,NetFlow,Median_age_years,Net_Migration_per_1000_of_Population,BranchCount,GroupName
84,1015560,2021-03-31,2021,"Industrial and Commercial Bank Of China Usa, N...",Community Bank,New York,NY,36061,New York,2913483000,2310614000,2516883000,396600000,CDR,4228000,0,4228000,-45583000,-0.114934,-0.015645,"Industrial and Commercial Bank Of China Usa, N...",602654,202 Canal Street,36061,New York,NY,36061,New York,NY,New York,New York,CDR,202 Canal Street,10013,1634989,2016,7884.0,36.7,4.822,8,group_1
85,1015560,2021-03-31,2021,"Industrial and Commercial Bank Of China Usa, N...",Community Bank,New York,NY,36061,New York,2913483000,2310614000,2516883000,396600000,CDR,4228000,0,4228000,-45583000,-0.114934,-0.015645,"Industrial and Commercial Bank Of China Usa, N...",602654,202 Canal Street,36061,New York,NY,36061,New York,NY,New York,New York,CDR,202 Canal Street,10013,1653877,2017,7947.0,37.1,4.805,8,group_1
86,1015560,2021-03-31,2021,"Industrial and Commercial Bank Of China Usa, N...",Community Bank,New York,NY,36061,New York,2913483000,2310614000,2516883000,396600000,CDR,4228000,0,4228000,-45583000,-0.114934,-0.015645,"Industrial and Commercial Bank Of China Usa, N...",602654,202 Canal Street,36061,New York,NY,36061,New York,NY,New York,New York,CDR,202 Canal Street,10013,1632480,2018,1770.0,37.3,1.084,8,group_1
87,1015560,2021-03-31,2021,"Industrial and Commercial Bank Of China Usa, N...",Community Bank,New York,NY,36061,New York,2913483000,2310614000,2516883000,396600000,CDR,4228000,0,4228000,-45583000,-0.114934,-0.015645,"Industrial and Commercial Bank Of China Usa, N...",602654,202 Canal Street,36061,New York,NY,36061,New York,NY,New York,New York,CDR,202 Canal Street,10013,1631993,2019,0.0,37.5,0.000,8,group_1
88,1015560,2021-03-31,2021,"Industrial and Commercial Bank Of China Usa, N...",Community Bank,New York,NY,36061,New York,2913483000,2310614000,2516883000,396600000,CDR,4228000,0,4228000,-45583000,-0.114934,-0.015645,"Industrial and Commercial Bank Of China Usa, N...",16967,"1633 Broadway, 28th Floor",36061,New York,NY,36061,New York,NY,New York,New York,CDR,"1633 Broadway, 28th Floor",10019,1634989,2016,7884.0,36.7,4.822,8,group_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9047,976703,2021-03-31,2021,"Bessemer Trust Company, National Association",Community Bank,New York,NY,36061,New York,2930954000,2539157000,2629540000,301415000,CDR,16152000,0,16152000,91384000,0.303183,0.031178,"Bessemer Trust Company, National Association",15153,630 Fifth Avenue,36061,New York,NY,36061,New York,NY,New York,New York,CDR,630 Fifth Avenue,10111,1631993,2019,0.0,37.5,0.000,4,group_1
9204,996260,2021-03-31,2021,Ctbc Bank Corp. (Usa),Community Bank,Los Angeles,CA,6037,Los Angeles,3969104000,3334802000,3399079000,570025000,CDR,12903000,0,12903000,40726000,0.071445,0.010260,Ctbc Bank Corp. (Usa),13219,"801 South Figueroa Street, Suite 2300",6037,Los Angeles,CA,6037,Los Angeles,CA,California,Los Angeles,CDR,"801 South Figueroa Street, Suite 2300",90017,10057155,2016,-22811.0,35.8,-2.268,28,group_1
9205,996260,2021-03-31,2021,Ctbc Bank Corp. (Usa),Community Bank,Los Angeles,CA,6037,Los Angeles,3969104000,3334802000,3399079000,570025000,CDR,12903000,0,12903000,40726000,0.071445,0.010260,Ctbc Bank Corp. (Usa),13219,"801 South Figueroa Street, Suite 2300",6037,Los Angeles,CA,6037,Los Angeles,CA,California,Los Angeles,CDR,"801 South Figueroa Street, Suite

In [29]:
banks_group2 = merged_data.loc[merged_data.GroupName == 'group_2']
banks_group2

,IDRSSD,Cycle_Date,Cycle Year,CU_Bank_Name,Bank Type,City_Name,State_Code_x,County_fips_x,County_Name_x,TOTAL_ASSETS,TOTAL_DEPOSITS,TOTAL_LIABILITIES,TOTAL_EQUITY_CAPITAL,Source_Type_x,Net_Income,a,Quaterly_Net_Income,Annual_Net_Income,ROE,ROA,CU_BANK_NAME,SiteId,Site_Name,County_fips_y,County_Name_y,State_Code_y,Branch_County_FIPS,Branch_County_Name,Branch_State_Code,Branch_State_Name,Branch_City_Name,Source_Type_y,PhysicalAddressLine1,PhysicalAddressPostalCode,Population,Year,NetFlow,Median_age_years,Net_Migration_per_1000_of_Population,BranchCount,GroupName
660,1216321,2021-03-31,2021,Republic Bank,Community Bank,Philadelphia,PA,42101,Philadelphia,5386210000,4385208000,5097955000,288255000,CDR,7612000,0,7612000,14964000,0.051912,0.002778,Republic Bank,479318,833 Chestnut Street Suite 170,42101,Philadelphia,PA,42101,Philadelphia,PA,Pennsylvania,Philadelphia,CDR,833 Chestnut Street Suite 170,19107,1559938,2016,834.0,33.9,0.534,28,group_2
661,1216321,2021-03-31,2021,Republic Bank,Community Bank,Philadelphia,PA,42101,Philadelphia,5386210000,4385208000,5097955000,288255000,CDR,7612000,0,7612000,14964000,0.051912,0.002778,Republic Bank,479318,833 Chestnut Street Suite 170,42101,Philadelphia,PA,42101,Philadelphia,PA,Pennsylvania,Philadelphia,CDR,833 Chestnut Street Suite 170,19107,1569657,2017,-408.0,34.1,-0.259,28,group_2
662,1216321,2021-03-31,2021,Republic Bank,Community Bank,Philadelphia,PA,42101,Philadelphia,5386210000,4385208000,5097955000,288255000,CDR,7612000,0,7612000,14964000,0.051912,0.002778,Republic Bank,479318,833 Chestnut Street Suite 170,42101,Philadelphia,PA,42101,Philadelphia,PA,Pennsylvania,Philadelphia,CDR,833 Chestnut Street Suite 170,19107,1575522,2018,3504.0,34.3,2.224,28,group_2
663,1216321,2021-03-31,2021,Republic Bank,Community Bank,Philadelphia,PA,42101,Philadelphia,5386210000,4385208000,5097955000,288255000,CDR,7612000,0,7612000,14964000,0.051912,0.002778,Republic Bank,479318,833 Chestnut Street Suite 170,42101,Philadelphia,PA,42101,Philadelphia,PA,Pennsylvania,Philadelphia,CDR,833 Chestnut Street Suite 170,19107,1579075,2019,0.0,34.4,0.000,28,group_2
664,1216321,2021-03-31,2021,Republic Bank,Community Bank,Philadelphia,PA,42101,Philadelphia,5386210000,4385208000,5097955000,288255000,CDR,7612000,0,7612000,14964000,0.051912,0.002778,Republic Bank,475503,8764 Frankford Avenue,42101,Philadelphia,PA,42101,Philadelphia,PA,Pennsylvania,Philadelphia,CDR,8764 Frankford Avenue,19136,1559938,2016,834.0,33.9,0.534,28,group_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8791,945978,2021-03-31,2021,POLICE & FIRE,Credit Union Bank,Philadelphia,PA,42101,Philadelphia,7117031958,5919922640,0,5919922640,CU,1128710671,0,1128710671,1195076934,0.201873,0.167917,POLICE & FIRE,15749,Main Office,42101,Philadelphia,PA,42101,Philadelphia,PA,Pennsylvania,Philadelphia,CU,901 Arch St,19107-2404,1579075,2019,0.0,34.4,0.000,32,group_2
8792,945978,2021-03-31,2021,POLICE & FIRE,Credit Union Bank,Philadelphia,PA,42101,Philadelphia,7117031958,5919922640,0,5919922640,CU,1128710671,0,1128710671,1195076934,0.201873,0.167917,POLICE & FIRE,50945,Mayfair Branch,42101,Philadelphia,PA,42101,Philadelphia,PA,Pennsylvania,Philadelphia,CU,7145 Frankford Ave,19135-1008,1559938,2016,834.0,33.9,0.534,32,group_2
8793,945978,2021-03-31,2021,POLICE & FIRE,Credit Union Bank,Philadelphia,PA,42101,Philadelphia,7117031958,5919922640,0,5919922640,CU,1128710671,0,1128710671,1195076934,0.201873,0.167917,POLICE & FIRE,50945,Mayfair Branch,42101,Philadelphia,PA,42101,Philadelphia,PA,Pennsylvania,Philadelphia,CU,7145 Frankford Ave,19135-1008,1569657,2017,-408.0,34.1,-0.259,32,group_2
8794,945978,2021-03-31,2021,POLICE & FIRE,Credit Union Bank,Philadelphia,PA,42101,Philadelphia,7117031958,5919922640,0,5919922640,CU,1128710671,0,1128710671,1195076934,0.201873,0.167917,POLICE & FIRE,50945,Mayfair Branch,42101,Philadelphia,PA,42101,Philadelphia,PA,

In [30]:
banks_group3 = merged_data.loc[merged_data.GroupName == 'group_3']
banks_group3

,IDRSSD,Cycle_Date,Cycle Year,CU_Bank_Name,Bank Type,City_Name,State_Code_x,County_fips_x,County_Name_x,TOTAL_ASSETS,TOTAL_DEPOSITS,TOTAL_LIABILITIES,TOTAL_EQUITY_CAPITAL,Source_Type_x,Net_Income,a,Quaterly_Net_Income,Annual_Net_Income,ROE,ROA,CU_BANK_NAME,SiteId,Site_Name,County_fips_y,County_Name_y,State_Code_y,Branch_County_FIPS,Branch_County_Name,Branch_State_Code,Branch_State_Name,Branch_City_Name,Source_Type_y,PhysicalAddressLine1,PhysicalAddressPostalCode,Population,Year,NetFlow,Median_age_years,Net_Migration_per_1000_of_Population,BranchCount,GroupName
20,1002878,2021-03-31,2021,Northwest Bank,Community Bank,Warren,PA,42123,Warren,14402674000,12435962000,12842643000,1560031000,CDR,42111000,0,42111000,114157000,0.073176,0.007926,Northwest Bank,265367,125 Ludlow Street,42123,Warren,PA,42123,Warren,PA,Pennsylvania,Warren,CDR,125 Ludlow Street,16365,40649,2016,-368.0,46.4,-9.053,72,group_3
21,1002878,2021-03-31,2021,Northwest Bank,Community Bank,Warren,PA,42123,Warren,14402674000,12435962000,12842643000,1560031000,CDR,42111000,0,42111000,114157000,0.073176,0.007926,Northwest Bank,265367,125 Ludlow Street,42123,Warren,PA,42123,Warren,PA,Pennsylvania,Warren,CDR,125 Ludlow Street,16365,40345,2017,-334.0,46.7,-8.278,72,group_3
22,1002878,2021-03-31,2021,Northwest Bank,Community Bank,Warren,PA,42123,Warren,14402674000,12435962000,12842643000,1560031000,CDR,42111000,0,42111000,114157000,0.073176,0.007926,Northwest Bank,265367,125 Ludlow Street,42123,Warren,PA,42123,Warren,PA,Pennsylvania,Warren,CDR,125 Ludlow Street,16365,40035,2018,-452.0,47.0,-11.290,72,group_3
23,1002878,2021-03-31,2021,Northwest Bank,Community Bank,Warren,PA,42123,Warren,14402674000,12435962000,12842643000,1560031000,CDR,42111000,0,42111000,114157000,0.073176,0.007926,Northwest Bank,265367,125 Ludlow Street,42123,Warren,PA,42123,Warren,PA,Pennsylvania,Warren,CDR,125 Ludlow Street,16365,39756,2019,0.0,47.4,0.000,72,group_3
24,1002878,2021-03-31,2021,Northwest Bank,Community Bank,Warren,PA,42123,Warren,14402674000,12435962000,12842643000,1560031000,CDR,42111000,0,42111000,114157000,0.073176,0.007926,Northwest Bank,265367,125 Ludlow Street,19041,Clay,IA,42123,Warren,PA,Iowa,Warren,CDR,125 Ludlow Street,16365,40649,2016,-368.0,46.4,-9.053,72,group_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8907,959395,2021-03-31,2021,THE GOLDEN 1,Credit Union Bank,Sacramento,CA,6067,Sacramento,17237843720,15425836415,0,15425836415,CU,1500895371,0,1500895371,1571456522,0.101871,0.091163,THE GOLDEN 1,50520,Consumnes,6067,Sacramento,CA,6067,Sacramento,CA,California,Sacramento,CU,8241 Bruceville Rd Ste D-100,95823-2359,1524553,2019,0.0,36.2,0.000,100,group_3
8908,959395,2021-03-31,2021,THE GOLDEN 1,Credit Union Bank,Sacramento,CA,6067,Sacramento,17237843720,15425836415,0,15425836415,CU,1500895371,0,1500895371,1571456522,0.101871,0.091163,THE GOLDEN 1,50509,Southgate,6067,Sacramento,CA,6067,Sacramento,CA,California,Sacramento,CU,4430 Florin Rd Ste 1,95823-2500,1479300,2016,14591.0,35.7,9.863,100,group_3
8909,959395,2021-03-31,2021,THE GOLDEN 1,Credit Union Bank,Sacramento,CA,6067,Sacramento,17237843720,15425836415,0,15425836415,CU,1500895371,0,1500895371,1571456522,0.101871,0.091163,THE GOLDEN 1,50509,Southgate,6067,Sacramento,CA,6067,Sacramento,CA,California,Sacramento,CU,4430 Florin Rd Ste 1,95823-2500,1495400,2017,16439.0,35.9,10.993,100,group_3
8910,959395,2021-03-31,2021,THE GOLDEN 1,Credit Union Bank,Sacramento,CA,6067,Sacramento,17237843720,15425836415,0,15425836415,CU,1500895371,0,1500895371,1571456522,0.101871,0.091163,THE GOLDEN 1,50509,Southgate,6067,Sacramento,CA,6067,Sacramento,CA,California,Sacramento,CU,4430 Florin Rd Ste 1,95823-2500,1510023,2018,17495.0,36.0,11.585,100,group_3


In [31]:
county_data_0 = banks_group0.groupby("County_Name_x").agg({'NetFlow': 'sum', 'Population': 'sum', 'Median_age_years': 'max'})
county_data_0.to_csv("county_data_group_0.csv")

In [32]:
county_data_1 = banks_group1.groupby("County_Name_x").agg({'NetFlow': 'sum', 'Population': 'sum', 'Median_age_years': 'max'})
county_data_1.to_csv("county_data_group_1.csv")

In [33]:
county_data_2 = banks_group2.groupby("County_Name_x").agg({'NetFlow': 'sum', 'Population': 'sum', 'Median_age_years': 'max'})
county_data_2.to_csv("county_data_group_2.csv")

In [34]:
county_data_3 = banks_group3.groupby("County_Name_x").agg({'NetFlow': 'sum', 'Population': 'sum', 'Median_age_years': 'max'})
county_data_3.to_csv("county_data_group_3.csv")

In [35]:
county_data_0 = pd.read_csv("county_data_group_0.csv")
county_data_dict_0 = county_data_0.set_index('County_Name_x').T.to_dict('list')
county_data_dict_0

{'Abbeville': [-242.0, 99023.0, 44.3],
 'Adair': [-3468.0, 114924.0, 46.0],
 'Aiken': [14112.0, 5328640.0, 41.0],
 'Aitkin': [-1200.0, 126462.0, 55.5],
 'Alamosa': [11058.0, 391494.0, 31.8],
 'Albany': [271450.0, 30771625.0, 37.9],
 'Allegan': [-3639.0, 459204.0, 40.0],
 'Anchorage': [0.0, 7170552.0, 33.1],
 'Anderson': [5014.0, 1561884.0, 40.8],
 'Andrews': [3724.0, 282584.0, 31.6],
 'Appomattox': [-2324.0, 124120.0, 43.5],
 'Arlington': [8666.0, 1841786.0, 34.7],
 'Ashland': [3031.0, 402787.0, 42.0],
 'Ashtabula': [-17318.0, 2756341.0, 42.5],
 'Atchison': [-3400.0, 262616.0, 36.6],
 'Athens': [71656.0, 1050076.0, 29.6],
 'Atoka': [410.0, 110886.0, 40.8],
 'Audubon': [-1350.0, 45424.0, 49.0],
 'Baltimore': [202446.0, 40956678.0, 39.4],
 'Bamberg': [241.0, 58875.0, 41.9],
 'Bandera': [3648.0, 86309.0, 52.2],
 'Barron': [-2248.0, 362774.0, 44.2],
 'Bastrop': [24104.0, 1302764.0, 38.8],
 'Beaver': [-3407.0, 735243.0, 45.1],
 'Bedford': [-1317.0, 194691.0, 46.5],
 'Bennington': [3148.0, 2

In [36]:
county_data_1 = pd.read_csv("county_data_group_1.csv")
county_data_dict_1 = county_data_1.set_index('County_Name_x').T.to_dict('list')
county_data_dict_1

{'Alexandria': [0.0, 1837098.0, 36.4],
 'Anchorage': [0.0, 8963190.0, 33.1],
 'Brookings': [66848.0, 2175552.0, 27.0],
 'Clearfield': [15810.0, 1930998.0, 45.0],
 'Dallas': [30252.0, 41034748.0, 33.4],
 'Danville': [0.0, 337084.0, 41.5],
 'Denver': [129943.0, 30148393.0, 34.5],
 'Eau Claire': [81952.0, 6572800.0, 34.5],
 'El Paso': [-191103.0, 110290389.0, 32.2],
 'Fresno': [149084.0, 50666551.0, 32.2],
 'Grand Forks': [37900.0, 1122188.0, 29.7],
 'Greenville': [35889.0, 5933784.0, 38.2],
 'Hartford': [-2561.0, 3581407.0, 40.4],
 'Honolulu': [220248.0, 82939878.0, 37.9],
 'Lafayette': [0.0, 3312351.0, 34.5],
 'Los Angeles': [-1229536.0, 564794986.0, 36.5],
 'Lubbock': [670491.0, 32372622.0, 30.8],
 'Manitowoc': [-8218.0, 636900.0, 44.9],
 'Martinsville': [0.0, 26807.0, 42.7],
 'Midland': [1510.0, 333792.0, 41.8],
 'Muskogee': [8994.0, 554460.0, 37.9],
 'New York': [246414.0, 91746746.0, 37.5],
 'Newport': [6163.0, 331794.0, 45.2],
 'Philadelphia': [62880.0, 100547072.0, 34.4],
 'Provid

In [37]:
county_data_2 = pd.read_csv("county_data_group_2.csv")
county_data_dict_2 = county_data_2.set_index('County_Name_x').T.to_dict('list')
county_data_dict_2

{'Arlington': [4333.0, 920893.0, 34.7],
 'Dallas': [105882.0, 143621618.0, 33.4],
 'Denver': [35439.0, 8222289.0, 34.5],
 'Effingham': [-2367.0, 410493.0, 39.4],
 'Honolulu': [325128.0, 122435058.0, 37.9],
 'Indiana': [135207.0, 3103461.0, 40.2],
 'Los Angeles': [-1141712.0, 524452487.0, 36.5],
 'New York': [88005.0, 32766695.0, 37.5],
 'Peoria': [-48986.0, 5181372.0, 37.5],
 'Philadelphia': [58950.0, 94262880.0, 34.4],
 'Racine': [-15978.0, 4685844.0, 40.2]}

In [38]:
county_data_3 = pd.read_csv("county_data_group_3.csv")
county_data_dict_3 = county_data_3.set_index('County_Name_x').T.to_dict('list')
county_data_dict_3

{'Anchorage': [0.0, 4780368.0, 33.1],
 'Arlington': [4333.0, 920893.0, 34.7],
 'Champaign': [194808.0, 5002416.0, 30.1],
 'Dallas': [22689.0, 30776061.0, 33.4],
 'Fairfax': [266580.0, 55337784.0, 39.6],
 'Lancaster': [60021.0, 19340550.0, 38.6],
 'Los Angeles': [-1141712.0, 524452487.0, 36.5],
 'New York': [774444.0, 288346916.0, 37.5],
 'Passaic': [-28144.0, 8101780.0, 37.2],
 'Portsmouth': [0.0, 574821.0, 35.2],
 'Sacramento': [727875.0, 90139140.0, 36.2],
 'San Diego': [271312.0, 210494832.0, 35.8],
 'San Juan': [0.0, 4313550.0, 41.8],
 'Spokane': [528502.0, 25740039.0, 37.6],
 'Tulsa': [249050.0, 43454482.0, 35.7],
 'Walla Walla': [17440.0, 961088.0, 37.5],
 'Warren': [-10386.0, 1447065.0, 47.4]}

In [39]:
# List
county_scores_0 = [ ['first', 0, 'row' , 0, 1,1]]#,1,1]]

           
#Create a DataFrame object
county_scores_df_0 = pd.DataFrame(county_scores_0, 
                  columns = ['County_1' , 'Attributes_1', 'County_2', 'Attributes_2', 
                             'Euclidean_Similarity_Score',
                             'Cosine_Similarity_Score',])
                             #'Manhattan_Similarity_Score',
                             #'Jaccard_Similarity_Score']) 
county_scores_df_0

,County_1,Attributes_1,County_2,Attributes_2,Euclidean_Similarity_Score,Cosine_Similarity_Score
0,first,0,row,0,1,1


In [40]:
# List
county_scores_1 = [ ['first', 1, 'row' , 1, 1,1]]#,1,1]]

           
#Create a DataFrame object
county_scores_df_1 = pd.DataFrame(county_scores_1, 
                  columns = ['County_1' , 'Attributes_1', 'County_2', 'Attributes_2', 
                             'Euclidean_Similarity_Score',
                             'Cosine_Similarity_Score',])
                             #'Manhattan_Similarity_Score',
                             #'Jaccard_Similarity_Score']) 
county_scores_df_1

,County_1,Attributes_1,County_2,Attributes_2,Euclidean_Similarity_Score,Cosine_Similarity_Score
0,first,1,row,1,1,1


In [41]:
# List
county_scores_2 = [ ['first', 2, 'row' , 2, 2,2]]#,2,2]]

           
#Create a DataFrame object
county_scores_df_2 = pd.DataFrame(county_scores_2, 
                  columns = ['County_1' , 'Attributes_1', 'County_2', 'Attributes_2', 
                             'Euclidean_Similarity_Score',
                             'Cosine_Similarity_Score',])
                             #'Manhattan_Similarity_Score',
                             #'Jaccard_Similarity_Score']) 
county_scores_df_2

,County_1,Attributes_1,County_2,Attributes_2,Euclidean_Similarity_Score,Cosine_Similarity_Score
0,first,2,row,2,2,2


In [42]:
# List
county_scores_3 = [ ['first', 3, 'row' , 3, 3,3]]#,3,3]]

           
#Create a DataFrame object
county_scores_df_3 = pd.DataFrame(county_scores_3, 
                  columns = ['County_1' , 'Attributes_1', 'County_2', 'Attributes_2', 
                             'Euclidean_Similarity_Score',
                             'Cosine_Similarity_Score',])
                             #'Manhattan_Similarity_Score',
                             #'Jaccard_Similarity_Score']) 
county_scores_df_3

,County_1,Attributes_1,County_2,Attributes_2,Euclidean_Similarity_Score,Cosine_Similarity_Score
0,first,3,row,3,3,3


In [ ]:
## county level similarity scores
for county_1, value_1 in tqdm(county_data_dict_0.items()):
    
    for county_2, value_2 in county_data_dict_0.items():
        score_list = []
    
        #To Calculate cosine similarity        
        eucl_score = distance.euclidean(value_1,value_2)
        cosine_score = 1- spatial.distance.cosine(value_1,value_2)
        #manhattan_score = manhattan_distance(np.array(value_1, dtype=float), np.array(value_2, dtype=float))
        #jaccard_score = jaccard_similarity(value_1,value_2)
        minowski_score = 1- spatial.distance.minkowski(value_1,value_2)
        list_1 = [county_1, value_1, county_2, value_2, eucl_score, cosine_score]#,manhattan_score,jaccard_score]
        county_scores_df_0.loc[len(county_scores_df_0)] = list_1
        

 15%|█▌        | 36/240 [00:31<03:06,  1.09it/s]

In [ ]:
## county level similarity scores
for county_1, value_1 in tqdm(county_data_dict_1.items()):
    
    for county_2, value_2 in county_data_dict_1.items():
        score_list = []
    
        #To Calculate cosine similarity        
        eucl_score = distance.euclidean(value_1,value_2)
        cosine_score = 1- spatial.distance.cosine(value_1,value_2)
        #manhattan_score = manhattan_distance(np.array(value_1, dtype=float), np.array(value_2, dtype=float))
        #jaccard_score = jaccard_similarity(value_1,value_2)
        minowski_score = 1- spatial.distance.minkowski(value_1,value_2)
        list_1 = [county_1, value_1, county_2, value_2, eucl_score, cosine_score]#,manhattan_score,jaccard_score]
        county_scores_df_1.loc[len(county_scores_df_1)] = list_1

In [ ]:
## county level similarity scores
for county_2, value_2 in tqdm(county_data_dict_2.items()):
    
    for county_2, value_2 in county_data_dict_2.items():
        score_list = []
    
        #To Calculate cosine similarity        
        eucl_score = distance.euclidean(value_2,value_2)
        cosine_score = 1- spatial.distance.cosine(value_2,value_2)
        #manhattan_score = manhattan_distance(np.array(value_2, dtype=float), np.array(value_2, dtype=float))
        #jaccard_score = jaccard_similarity(value_2,value_2)
        minowski_score = 1- spatial.distance.minkowski(value_2,value_2)
        list_2 = [county_1, value_1, county_2, value_2, eucl_score, cosine_score]#,manhattan_score,jaccard_score]
        county_scores_df_2.loc[len(county_scores_df_2)] = list_2
        

In [ ]:
## county level similarity scores
for county_3, value_3 in tqdm(county_data_dict_3.items()):
    
    for county_3, value_3 in county_data_dict_3.items():
        score_list = []
    
        #To Calculate cosine similarity        
        eucl_score = distance.euclidean(value_3,value_3)
        cosine_score = 1- spatial.distance.cosine(value_3,value_3)
        #manhattan_score = manhattan_distance(np.array(value_3, dtype=float), np.array(value_3, dtype=float))
        #jaccard_score = jaccard_similarity(value_3,value_3)
        minowski_score = 1- spatial.distance.minkowski(value_3,value_3)
        list_3 = [county_1, value_1, county_2, value_2, eucl_score, cosine_score]#,manhattan_score,jaccard_score]
        county_scores_df_3.loc[len(county_scores_df_3)] = list_3
        

In [ ]:
county_scores_df_0
county_scores_df_0['Score_bucket'] = pd.qcut(county_scores_df_0['Euclidean_Similarity_Score'], 3,
                               labels = False)   # 0 = High, 1 = Med, 2 = Low

In [ ]:
county_scores_df_1
county_scores_df_1['Score_bucket'] = pd.qcut(county_scores_df_1['Euclidean_Similarity_Score'], 3,
                               labels = False)   # 1 = High, 1 = Med, 2 = Low

In [ ]:
county_scores_df_2
county_scores_df_2['Score_bucket'] = pd.cut(county_scores_df_2['Euclidean_Similarity_Score'], 3,
                               labels = False)   # 2 = High, 2 = Med, 2 = Low

In [ ]:
county_scores_df_3

county_scores_df_3['Score_bucket'] = pd.cut(county_scores_df_3['Euclidean_Similarity_Score'], 3,
                               labels = False)   # 3 = High, 3 = Med, 3 = Low

In [ ]:
county_scores_df_0.to_csv("county_scores_final_group_0.csv")

In [ ]:
county_scores_df_1.to_csv("county_scores_final_group_1.csv")

In [ ]:
county_scores_df_2.to_csv("county_scores_final_group_2.csv")

In [ ]:
county_scores_df_3.to_csv("county_scores_final_group_3.csv")

In [ ]:
banks_group0_crossjoin = banks_group0
banks_group0_crossjoin = banks_group0_crossjoin.drop(['NetFlow', 'Population', 'Median_age_years', 'Net_Migration_per_1000_of_Population',"State_Code_x"], axis = 1)

## Cross join
# importing pandas module


# Now to perform cross join, we will create
# a key column in both the DataFrames to
# merge on that key.
banks_group0_crossjoin['key'] = 1


# to obtain the cross join we will merge
# on the key and drop it.
banks_group0_crossjoin_final = pd.merge(banks_group0_crossjoin, banks_group0_crossjoin, on ='key').drop("key", 1)



## Join similarity score

joined__on_2_cond_table = banks_group0_crossjoin_final.merge(county_scores_df, how='inner', left_on=['County_x', 'County_y'], right_on=['County_1', 'County_2'])

## Remove unwanted columns
joined__on_2_cond_table = joined__on_2_cond_table.drop(['County_1', 'Attributes_1', 'County_2', 'Attributes_2',"Cosine_Similarity_Score"
                                                        #, 'Manhattan_Similarity_Score', 'Jaccard_Similarity_Score'
                                                        ,'Euclidean_Similarity_Score'], axis = 1)


In [ ]:
banks_group0_crossjoin = banks_group0
banks_group0_crossjoin = banks_group0_crossjoin.drop(['NetFlow', 'Population', 'Median_age_years', 'Net_Migration_per_1000_of_Population',"State_Code_x"], axis = 1)

## Cross join
# importing pandas module


# Now to perform cross join, we will create
# a key column in both the DataFrames to
# merge on that key.
banks_group0_crossjoin['key'] = 1


# to obtain the cross join we will merge
# on the key and drop it.
banks_group0_crossjoin_final = pd.merge(banks_group0_crossjoin, banks_group0_crossjoin, on ='key').drop("key", 1)



## Join similarity score

joined__on_2_cond_table_group0 = banks_group0_crossjoin_final.merge(county_scores_df, how='inner', left_on=['County_x', 'County_y'], right_on=['County_1', 'County_2'])

## Remove unwanted columns
joined__on_2_cond_table_group0 = joined__on_2_cond_table.drop(['County_1', 'Attributes_1', 'County_2', 'Attributes_2',"Cosine_Similarity_Score"
                                                        #, 'Manhattan_Similarity_Score', 'Jaccard_Similarity_Score'
                                                        ,'Euclidean_Similarity_Score'], axis = 1)


In [ ]:
banks_group1_crossjoin = banks_group1
banks_group1_crossjoin = banks_group1_crossjoin.drop(['NetFlow', 'Population', 'Median_age_years', 'Net_Migration_per_1000_of_Population',"State_Code_x"], axis = 1)

## Cross join
# importing pandas module


# Now to perform cross join, we will create
# a key column in both the DataFrames to
# merge on that key.
banks_group1_crossjoin['key'] = 1


# to obtain the cross join we will merge
# on the key and drop it.
banks_group1_crossjoin_final = pd.merge(banks_group1_crossjoin, banks_group1_crossjoin, on ='key').drop("key", 1)



## Join similarity score

joined__on_2_cond_table_group1 = banks_group1_crossjoin_final.merge(county_scores_df, how='inner', left_on=['County_x', 'County_y'], right_on=['County_1', 'County_2'])

## Remove unwanted columns
joined__on_2_cond_table_group1 = joined__on_2_cond_table.drop(['County_1', 'Attributes_1', 'County_2', 'Attributes_2',"Cosine_Similarity_Score"
                                                        #, 'Manhattan_Similarity_Score', 'Jaccard_Similarity_Score'
                                                        ,'Euclidean_Similarity_Score'], axis = 1)
banks_group1_crossjoin = banks_group1
banks_group1_crossjoin = banks_group1_crossjoin.drop(['NetFlow', 'Population', 'Median_age_years', 'Net_Migration_per_1000_of_Population',"State_Code_x"], axis = 1)

## Cross join
# importing pandas module


# Now to perform cross join, we will create
# a key column in both the DataFrames to
# merge on that key.
banks_group1_crossjoin['key'] = 1


# to obtain the cross join we will merge
# on the key and drop it.
banks_group1_crossjoin_final = pd.merge(banks_group1_crossjoin, banks_group1_crossjoin, on ='key').drop("key", 1)



## Join similarity score

joined__on_2_cond_table_group0 = banks_group1_crossjoin_final.merge(county_scores_df, how='inner', left_on=['County_x', 'County_y'], right_on=['County_1', 'County_2'])

## Remove unwanted columns
joined__on_2_cond_table_group0 = joined__on_2_cond_table.drop(['County_1', 'Attributes_1', 'County_2', 'Attributes_2',"Cosine_Similarity_Score"
                                                        #, 'Manhattan_Similarity_Score', 'Jaccard_Similarity_Score'
                                                        ,'Euclidean_Similarity_Score'], axis = 1)


In [ ]:
banks_group2_crossjoin = banks_group2
banks_group2_crossjoin = banks_group2_crossjoin.drop(['NetFlow', 'Population', 'Median_age_years', 'Net_Migration_per_1000_of_Population',"State_Code_x"], axis = 1)

## Cross join
# importing pandas module


# Now to perform cross join, we will create
# a key column in both the DataFrames to
# merge on that key.
banks_group2_crossjoin['key'] = 1


# to obtain the cross join we will merge
# on the key and drop it.
banks_group2_crossjoin_final = pd.merge(banks_group2_crossjoin, banks_group2_crossjoin, on ='key').drop("key", 1)



## Join similarity score

joined__on_2_cond_table_group2 = banks_group2_crossjoin_final.merge(county_scores_df, how='inner', left_on=['County_x', 'County_y'], right_on=['County_1', 'County_2'])

## Remove unwanted columns
joined__on_2_cond_table_group2 = joined__on_2_cond_table.drop(['County_1', 'Attributes_1', 'County_2', 'Attributes_2',"Cosine_Similarity_Score"
                                                        #, 'Manhattan_Similarity_Score', 'Jaccard_Similarity_Score'
                                                        ,'Euclidean_Similarity_Score'], axis = 1)


In [ ]:
banks_group3_crossjoin = banks_group3
banks_group3_crossjoin = banks_group3_crossjoin.drop(['NetFlow', 'Population', 'Median_age_years', 'Net_Migration_per_1000_of_Population',"State_Code_x"], axis = 1)

## Cross join
# importing pandas module


# Now to perform cross join, we will create
# a key column in both the DataFrames to
# merge on that key.
banks_group3_crossjoin['key'] = 1


# to obtain the cross join we will merge
# on the key and drop it.
banks_group3_crossjoin_final = pd.merge(banks_group3_crossjoin, banks_group3_crossjoin, on ='key').drop("key", 1)



## Join similarity score

joined__on_2_cond_table_group3 = banks_group3_crossjoin_final.merge(county_scores_df, how='inner', left_on=['County_x', 'County_y'], right_on=['County_1', 'County_2'])

## Remove unwanted columns
joined__on_2_cond_table_group3 = joined__on_2_cond_table.drop(['County_1', 'Attributes_1', 'County_2', 'Attributes_2',"Cosine_Similarity_Score"
                                                        #, 'Manhattan_Similarity_Score', 'Jaccard_Similarity_Score'
                                                        ,'Euclidean_Similarity_Score'], axis = 1)
